In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# DATA LOAD

In [2]:
report = pd.read_excel('data.xlsx', sheet_name = 'Report', skiprows=1)

In [3]:
to_drop = ['Социальная сеть', 'Владелец']
report.drop(columns = to_drop, inplace = True)

In [4]:
report.head()

,ID поста,Текст,Эмоциональный окрас,Дата,Комментариев,Лайков,Репостов,Просмотров,Негативных комментариев,Позитивных комментариев,Нейтральных комментариев,Неопределенных комментариев,Вежливых комментариев,Юмористических комментариев
0,-159668210_25795,Нам задают много вопросов о технологическом ко...,Нейтральность,2021-12-31 07:45:00,0,0,0,3548.0,0,0,0,0,0,0
1,-74188185_20318,В высадке на Марс бизнесмен видит возможность ...,Нейтральность,2021-12-31 07:41:31,0,2,0,346.0,0,0,0,0,0,0
2,-74188185_20317,Гораздо чаще остальных американские компании и...,Нейтральность,2021-12-31 07:40:42,0,0,0,319.0,0,0,0,0,0,0
3,-21732035_11926,"Twitter-аккаунт, который ранее связывали с защ...",Негатив,2021-12-31 07:23:16,0,2,0,222.0,0,0,0,0,0,0
4,-718901_3742,Дорогие коллеги и друзья! \n⠀ \nСпасибо за ваш...,Позитив,2021-12-31 07:13:35,0,37,2,3956.0,0,0,0,0,0,0


In [5]:
report.shape

(13630, 14)

In [6]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
import numpy as np
import pymorphy2
import nltk
morph = pymorphy2.MorphAnalyzer()
stopwords_ru = stopwords.words("russian")

In [7]:
data = report.Текст.values.tolist()

# 1 этап

In [14]:
data = report.Текст.values.tolist()

# 2 этап

In [15]:
data = [re.sub('\s+', ' ', sent) for sent in data] # убираем new line characters

In [16]:
data = [re.sub("\d+", ' ', sent) for sent in data] # убираем всё цифры

In [17]:
data = [re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", ' ', sent) for sent in data] # убираем url

# Левая ветка

# 3 этап левая ветка

In [18]:
data_left = [re.sub(r'[^\w\s]','', sent) for sent in data] # убираем знаки препинания.

## 4.1 этап левая ветка (нормализация)

In [19]:
data_left_norm = []  # с нормализацией
for i in tqdm(range(len(data_left))):
    data_left_norm.append(list(morph.parse(token)[0].normal_form for token in word_tokenize(data_left[i].lower().strip(), language="russian")))

100%|████████████████████████████████████████████████████████████████████████████| 13630/13630 [02:50<00:00, 79.97it/s]


## 4.2 этап левая ветка

In [20]:
data_left_d= []        # без нормализации
for i in tqdm(range(len(data_left))):
    data_left_d.append(word_tokenize(data_left[i].lower().strip(), language="russian"))

100%|██████████████████████████████████████████████████████████████████████████| 13630/13630 [00:03<00:00, 3474.75it/s]


## 5.1 этап левая ветка (нормализация)

In [21]:
bi_gram_norm = []
for sentence in tqdm(data_left_norm): # по постам
    data_left_norm_stop = []
    for word in sentence:                # по словам
        if word in stopwords_ru:
            continue
        else:
            data_left_norm_stop.append(word)
    bi_gram_norm += list(nltk.bigrams(data_left_norm_stop))

100%|█████████████████████████████████████████████████████████████████████████| 13630/13630 [00:01<00:00, 10892.84it/s]


## 5.2 этап левая ветка

In [22]:
bi_gram = []
for sentence in tqdm(data_left_d): # по постам
    data_left_d_stop = []
    for word in sentence:                # по словам
        if word in stopwords_ru:
            continue
        else:
            data_left_d_stop.append(word)
    bi_gram += list(nltk.bigrams(data_left_d_stop))

100%|██████████████████████████████████████████████████████████████████████████| 13630/13630 [00:01<00:00, 9808.69it/s]


## 6.1 этап левая ветка (нормализация)

In [23]:
bgfd = nltk.FreqDist(bi_gram_norm) # нормализованы
bgfd.most_common(10)

[(('принять', 'участие'), 361),
 (('искусственный', 'интеллект'), 328),
 (('это', 'год'), 315),
 (('технополис', 'москва'), 309),
 (('база', 'данные'), 285),
 (('информационный', 'безопасность'), 261),
 (('оэз', 'технополис'), 258),
 (('язык', 'программирование'), 243),
 (('подать', 'заявка'), 228),
 (('машинный', 'обучение'), 218)]

## 6.2 этап левая ветка

In [24]:
bgfd1 = nltk.FreqDist(bi_gram) # не нормализованы
bgfd1.most_common(10)

[(('технополис', 'москва'), 306),
 (('оэз', 'технополис'), 258),
 (('принять', 'участие'), 221),
 (('информационной', 'безопасности'), 214),
 (('data', 'science'), 202),
 (('многое', 'другое'), 198),
 (('blockchain', 'life'), 196),
 (('подать', 'заявку'), 167),
 (('нашем', 'сайте'), 161),
 (('призовой', 'фонд'), 159)]

# Правая ветка

# 3 этап правая ветка

In [25]:
data_right = ' '.join(data)
data_right = sent_tokenize(data_right, language="russian")  # по предложениям

In [26]:
data_right = [re.sub(r'[^\w\s]','', sent) for sent in data_right] # убираем знаки препинания.

## 4.1 этап правая ветка (нормализация)

In [27]:
data_right_norm = []  # с нормализацией
for i in tqdm(range(len(data_right))):
    data_right_norm.append(list(morph.parse(token)[0].normal_form for token in word_tokenize(data_right[i].lower().strip(), language="russian")))

100%|███████████████████████████████████████████████████████████████████████████| 63262/63262 [02:57<00:00, 356.15it/s]


## 4.2 этап правая ветка

In [28]:
data_right_d = []        # без нормализации
for i in tqdm(range(len(data_right))):
    data_right_d.append(word_tokenize(data_right[i].lower().strip(), language="russian"))

100%|██████████████████████████████████████████████████████████████████████████| 63262/63262 [00:06<00:00, 9650.49it/s]


## 5.1 этап правая ветка (нормализация)

In [29]:
bi_gram_norm = []
for sentence in tqdm(data_right_norm): # по предложениям
    data_right_norm_stop = []
    for word in sentence:                # по словам
        if word in stopwords_ru:
            continue
        else:
            data_right_norm_stop.append(word)
    bi_gram_norm += list(nltk.bigrams(data_right_norm_stop))

100%|█████████████████████████████████████████████████████████████████████████| 63262/63262 [00:01<00:00, 47780.95it/s]


## 5.2 этап правая ветка

In [30]:
bi_gram_d = []
for sentence in tqdm(data_right_d): # по предложениям
    data_right_norm_stop = []
    for word in sentence:                # по словам
        if word in stopwords_ru:
            continue
        else:
            data_right_norm_stop.append(word)
    bi_gram_d += list(nltk.bigrams(data_right_norm_stop))

100%|█████████████████████████████████████████████████████████████████████████| 63262/63262 [00:01<00:00, 46700.93it/s]


## 6.1 этап правая ветка (нормализация)

In [31]:
bgfd = nltk.FreqDist(bi_gram_norm) # нормализованы
bgfd.most_common(10)

[(('принять', 'участие'), 361),
 (('искусственный', 'интеллект'), 328),
 (('это', 'год'), 314),
 (('технополис', 'москва'), 309),
 (('база', 'данные'), 285),
 (('информационный', 'безопасность'), 261),
 (('оэз', 'технополис'), 258),
 (('язык', 'программирование'), 243),
 (('подать', 'заявка'), 228),
 (('машинный', 'обучение'), 218)]

# 6.2 этап правая ветка

In [33]:
bgfd = nltk.FreqDist(bi_gram_d) # не нормализованы
bgfd.most_common(10)

[(('технополис', 'москва'), 306),
 (('оэз', 'технополис'), 258),
 (('принять', 'участие'), 221),
 (('информационной', 'безопасности'), 214),
 (('data', 'science'), 202),
 (('многое', 'другое'), 198),
 (('blockchain', 'life'), 196),
 (('подать', 'заявку'), 167),
 (('нашем', 'сайте'), 161),
 (('призовой', 'фонд'), 159)]